In [1]:
import importlib
import os
from models import *
from dataprocessor import *
def reload_custom_libs():
    import loss_functions
    import models
    import dataprocessor
    importlib.reload(loss_functions)
    importlib.reload(models)
    importlib.reload(dataprocessor)
    
folder_path = 'data'
files = [f for f in os.listdir(folder_path) if f.endswith('.xlsx')]
files = [file for file in files if file not in ("CC00.NYB.xlsx", "LB00.CME.xlsx", "ES00.CME.xlsx", "NQ00.CME.xlsx", "YM00.CBT.xlsx", "SP00.CME.xlsx")]
data_list = []
macd_timescales = [(8, 24), (16, 28), (32, 96)]
rtn_timescales = [1, 21, 63, 126, 252]
timesteps = 126

In [2]:
data_list = process_data_list(files, macd_timescales, rtn_timescales, test = True)
target_set, labels, map = generate_tensors(data_list, timesteps, encoder_type = "one-hot", return_map=True)
context_set, _ = generate_tensors(data_list, timesteps, encoder_type = "one-hot", contain_next_day_rtn=True)

生成张量, 并对类别信息进行on-hot 编码: 100%|██████████| 11/11 [00:00<00:00, 17.05it/s]


one-hot 编码中...
123456


生成张量, 并对类别信息进行on-hot 编码: 100%|██████████| 11/11 [00:00<00:00, 14.97it/s]


one-hot 编码中...
123456


In [2]:
batch_size = len(files)
# data_set = data_binder(context_set, target_set, labels, batch_size=batch_size)
# data_set.save(f"saved_data")

dataset = tf.data.Dataset.load("saved_data")

In [11]:
for data in dataset.take(1):
    print(data[-3])

tf.Tensor(
[[[-0.02426526 -0.00834106  0.00373832 ...  0.00860495  0.01085832
   -0.03964194]
  [ 0.01181394  0.01343928  0.01625819 ...  0.01186378  0.01110821
    0.01197581]]

 [[-0.02229623  0.01069266 -0.01132853 ...  0.01104003  0.01030717
   -0.02292929]
  [ 0.01235004  0.01148805  0.01509915 ...  0.00942288  0.00841007
    0.00844778]]

 [[ 0.0006398   0.00063939  0.         ... -0.0005305   0.06900212
   -0.06405164]
  [ 0.00789153  0.00703347  0.0062768  ...  0.04067121  0.03627074
    0.03232757]]

 ...

 [[-0.01050584 -0.00117971  0.01968504 ...  0.          0.01169293
    0.0040201 ]
  [ 0.02147215  0.0191907   0.01892566 ...  0.01547339  0.0148404
    0.01326044]]

 [[-0.02068274 -0.00387657 -0.02319427 ...  0.05082689 -0.00077967
   -0.01193538]
  [ 0.02203394  0.02075571  0.02023506 ...  0.01495489  0.01641433
    0.02494243]]

 [[-0.01510157 -0.00965131  0.01579692 ...  0.00991926  0.02501142
   -0.01253482]
  [ 0.01453195  0.02011096  0.01892758 ...  0.01670112  0.015

In [3]:
reload_custom_libs()
from models import *
from dataprocessor import *
from loss_functions import *

hidden_dim = 64 # 128
warm_up = 63
target_std = tf.cast(5e-2, tf.float64)
features_len = len(macd_timescales) + len(rtn_timescales)
encoding_size = len(files) + 1
x_shape = (batch_size, timesteps, features_len)
s_shape = (batch_size, timesteps, encoding_size)

model = ModelWrapper(features_len, hidden_dim, encoding_size, num_heads = 4)
model.build((x_shape, s_shape))

In [4]:
def train(batch_num: int, num_epochs: int):
    
    optimizer = tf.keras.optimizers.Adam(learning_rate=1e-3)
    dataset = tf.data.Dataset.load("saved_data").shuffle(buffer_size=10000).batch(batch_num)
    
    for epoch in range(num_epochs):
        iter_count = 0
        for batch_data in tqdm(dataset, desc=f"训练中...epoch{epoch}"):
            # 对每一个批次进行处理
            x_c_b, x_c_rtn_b, s_c_b, x_b, s_b, rtn_std_b, _, _ = batch_data
            with tf.GradientTape() as tape:
                losses = []
                mles = []
                sharpes = []
                # 遍历批次中的每一个样本
                for i in range(batch_num):
                    x_c, x_c_rtn, s_c, x, s, rtn_std = x_c_b[i], x_c_rtn_b[i], s_c_b[i], x_b[i], s_b[i], rtn_std_b[i]
                    result = model(x_c, x_c_rtn, s_c, x, s)
                    loss, mle, sharpe = joint_loss_function(result, rtn_std, target_std, warm_up, alpha=1)
                    losses.append(loss)
                    mles.append(mle)
                    sharpes.append(sharpe)

                # 计算批次的平均损失
                batch_loss = tf.reduce_mean(losses)
                batch_mle = tf.reduce_mean(mles)
                batch_sharpe = tf.reduce_mean(sharpes)

            grads = tape.gradient(batch_loss, model.trainable_variables)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            
            iter_count += 1
            if iter_count % 1 == 0:
                print(f"Epoch {epoch+1}/{num_epochs}, Iteration {iter_count}, Loss: {batch_loss.numpy()}, MLE: {batch_mle.numpy()}, Sharpe: {batch_sharpe.numpy()}")

        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {batch_loss.numpy()}")

result = train(batch_num = 126, num_epochs = 100)


训练中...epoch0:   0%|          | 0/23 [00:25<?, ?it/s]


KeyboardInterrupt: 